# *GMOnotebook*
#### **Notebook template for applying routine hyperspectral/segmentation cross-analysis phenomics workflow over new datasets**

###### Notebook template v0.2.03 (Oct 14, 2020)
###### v0.2.0 onward applies the Python `GMOdetector` package instead of the deprecated R `GMOdetectoR` package, for much faster runtime
###### v0.2.01 has calls to `conda` cleaned up with `conda activate` and `conda deactivate` in the same code blocks to avoid any problems with warnings/errors preventing automatic execution of subsequent blocks
###### v0.2.02 is cleaned up using path variables (declared in notebook) instead of hardcoded paths, for portability
###### v0.2.03 is more clean to avoid running alignment-processed images through `inference.py` if an analysis if being repeated... also has better instructions

<img src="Figures/WorkflowFlowchart.png">

In this workflow, images taken with the macroPhor Array dual RGB/hyperspectral imaging platform are analyzed by a workflow in which regression quantifies fluorescent signals in hyperspectral images, deep learning segments RGB images into different tissues, and these datasets are cross-referenced to produce statistics on growth of transgenic callus and shoot.

#### Instructions for use
1.  Enter information for the experiment below
2. Set <font color=blue>variables</font> for data paths and parameters, as instructed by colored boxes.
3. "Save as" with filename describing experiment and anything special about this analysis (e.g. T18_OD_TAO_wk7_automation_test_attempt2.ipynb)
4. Run notebook from console, by either one of two methods:
- In the top bar of JupyterLab, select "Run" and "Run all cells"
- In a terminal, enter the below command with the notebook filename inserted<br>
```jupyter nbconvert --to HTML --ExecutePreprocessor.timeout=-1 --allow-errors --execute insert_filename_here```
5. Wait for email

#### Experiment ID and quick description:

<div class="alert alert-block alert-success">
Provide a short description of the experiment in the below box. This should include unique identifier codes for the experiment, along with a short description of genotypes and treatments studied. The timepoint should also be included. </div>

#### Parameters for analysis:

<div class="alert alert-block alert-success">
The below variables must be modified appropriately every time this workflow is run over new images.
</div>

##### Data location
The `data` variable below provides the **complete** path to the folder containing data to be analyzed. This should include all folders and subfolders in which the data of interest is organized by. For the organizational system used for our lab's data, this should follow the format "/Experiment/Subexperiment/Timepoint/"

In [2]:
data_prefix="/run/media/labgroup/Elements_7/"

(base) 

: 1

In [3]:
data="${data_prefix}/GFP_versions/GFPA_cottonwoods/wk7/Fluorescent/"

(base) 

: 1

##### Sample information
Every experiment has a randomization datasheet, which was used to organize treatment and genotype information for each plate, prepare labels, and randomize plates. The path to this file is provided through the `randomization_datasheets` variable below. This workflow requires this datasheet in order to know which plates have which genotype/treatment. At a later date, we will integrate an ability to read this data directly from labels.

In [4]:
randomization_datasheet="/home/labgroup/data/GREGSCORNER/GFPA_cottonwoods/GFPA_randomized_list_MNrevised.xlsx"

(base) 

: 1

##### Exclusion of missing/contaminated explants from analysis

Set the `missing_explants` variable to `"Automatic"` if using model to automatically detect missing and contaminated explants. Note that this model is only supported for plates with 12 explants. Otherwise, provide an appropriately formatted `.csv` spreadsheet (see example)  of manually scored contamination / missing explant data.

In [5]:
missing_explants="Automatic"

(base) 

: 1

Enter your email where results will be sent

In [6]:
email=michael.nagle@oregonstate.edu

(base) 

: 1

<div class="alert alert-block alert-warning">
All variables below should be modified only as needed to indicate the fluorescent proteins in samples and the grid layout of explants. </div>

##### Provide information pertinent to the fluorophores in the sample

Below should be all known fluorescent components contained in the sample. This includes each fluorescent protein, as well as a "noise" or "diffraction" term if applicable. All of these components must exist in the user's spectral library. `GMOdetector` currently comes with a spectral library that includes, by default:
- DsRed
- ZsYellow
- GFP
- Chl
- ChlA
- ChlB
- Diffraction

In [7]:
fluorophores=(GFP Chl Diffraction) # An explanation of array variables in bash is here: https://tldp.org/LDP/Bash-Beginners-Guide/html/sect_10_02.html

(base) 

: 1

The user has the option of limiting loading of hyperspectral data and subsequent regression to a specific range of wavelengths, using the `desired_wavelength_range` array variable. This range should cover all fluorophores provided in the `fluorophores` array variable able. Aside from runtime, there is no disadvantage to including a wider range than is needed.

In [8]:
desired_wavelength_range=(500 900)

(base) 

: 1

##### To assist user inspection of regression results, false color plots will be produced by `GMOdetector` to show results of regression over whole samples. 
###### Note: In v0.2.0 of the workflow, these parameters are independent of those used later by `GMOlabeler` to produce by-explant plots including false color plots. These will be made the same in a later update.
The `FalseColor_channels` array variable indicates the components to be plotted as red, green and blue, in that order.

In [9]:
FalseColor_channels=(Chl GFP Diffraction)

(base) 

: 1

The `FalseColor_caps` array variable indicates an upper limit of signal for each of these component. Any signal at or above these values will appear with maximum brightness; thus, these variables are comparable to exposure on a camera. If caps are too high, not much signal at lower ranges will be seen. If cap for a given component is too low, the false color images will appear overexposed with respect to the component.

In [10]:
FalseColor_caps=(200 400 50)

(base) 

: 1

##### `GMOlabeler` will use the below parameters to classify individual explants on plates as transgenic or not.

In [11]:
reporter=GFP
#reporter=DsRed

(base) (base) 

: 1

Parameters for reporter signal threshold and pixel threshold must be provided by the user. Our grid search yielded several noted below. These were most recently calculated from statistics produced with Python `GMOdetector`.

<img src="Figures/GMOlabeler_parameters.png">

In [12]:
reporter_threshold=38
pixel_threshold=3

(base) (base) 

: 1

##### Grid information

Currently, `GMOlabeler` distinguishes explants by their expected position on a plate. This is supported for plates with 12 or 20 explants, arranged in specific positions on a grid. False positives may result in cases in which one explant grows to such an extent that it intrudes into the grid square meant for an adjacent explant. <br>In the future, an additional neural network will be used to segment individual explants on a per-pixel basis, avoiding the need for crude cropping, and the outputs from the resulting model will be integrated with `GMOlabeler`.

Cropping to grid squares is supported for standard grids with 12 or 20 explants, as indicated by the `grid` variable. An image of each grid square (with a file path provided by the `grid_file` variable) will be superimposed over each image during `GMOlabeler` analysis to allow the user to easily inspect and verify grid positions.

In [13]:
grid=12

(base) 

: 1

<div class="alert alert-block alert-danger">
The below variables do not need to be modified during any routine use of the workflow.
</div>

For the following variables, use 0 (False) or 1 (True)

In [14]:
composite=1

(base) 

: 1

Set dimensions for plot outputs

In [15]:
#width=15
width=9
height=5

(base) (base) (base) 

: 1

In [16]:
parallel=0

(base) 

: 1

#### Paths to workflow components on local machine

In [17]:
cwd="/home/labgroup/code/GMOnotebook"

(base) 

: 1

In [18]:
#gmodetector_wd="/scratch2/NSF_GWAS/notebooks/gmodetector_py/"
gmodetector_wd="/home/labgroup/code/gmodetector_py/"
spectral_library_path="${gmodetector_wd}spectral_library/"
#output_directory_prefix="/scratch2/NSF_GWAS/gmodetector_out/"
#deeplab_path="/scratch2/NSF_GWAS/deeplab/"
deeplab_path="/home/labgroup/code/deeplab/"
#alignment_path="/scratch2/NSF_GWAS/ImageAlignment/"
alignment_path="/home/labgroup/code/ImageAlignment/"
#gmolabeler_path="/scratch2/NSF_GWAS/GMOlabeler/"
gmolabeler_path="/home/labgroup/code/GMOlabeler/"
#contamination_path="/scratch2/NSF_GWAS/Contamination"
contamination_path="/home/labgroup/code/DenseNet"
data_prefix="/home/labgroup/data/"
output_directory_prefix="${data_prefix}gmodetector_out/"

(base) (base) (base) (base) (base) (base) (base) (base) (base) (base) (base) (base) (base) (base) 

: 1

In [19]:
#grid_file="/scratch2/NSF_GWAS/macroPhor_Array/grids/grid20_post_processed.png"
#grid_file="/scratch2/NSF_GWAS/macroPhor_Array/grids/grids_left_facing_125208_1_0_1_rgb_processed.jpg"

# For data from before when imager went off-center
# grid_file="${gmolabeler_path}/grids/grid_12_explants_left-facing_processed.jpg"

# For offcenter images
grid_file="${alignment_path}/output/12itemoffcenterleftfacinga3_F1.9_I5.0_L80_cyan_172334_0_0_0_rgb_transformed_rgb.jpg"

(base) (base) (base) (base) (base) (base) (base) (base) 

: 1

<div class="alert alert-block alert-info">
With all above variables set, please "Save as..." with a filename referencing this specific dataset. <br>Finally, deploy the workflow (Step 4 in above instructions).
</div>

# Automated workflow to be deployed

See the below code for a walkthrough of how GMOnotebook works, or view the outputs after running the workflow for help troubleshooting errors in specific steps of analysis.

<div class="alert alert-block alert-danger"> <b>Danger:</b> Do not modify any below code without creating a new version of the template notebook. During routine usage, this workflow should be customized only by modifying variables above, while leaving the below code unmodified. </div>

These internal variables are set automatically.

In [22]:
datestamp=$(date +”%Y-%m-%d”)

In [23]:
data_folder=$(echo $data | cut -d/ -f5-)

In [24]:
timepoint="$(basename -- $data_folder)"

In [25]:
output_directory_full="$output_directory_prefix$data_folder"

In [26]:
dataset_name=$(echo $subdirectory | sed -e 's/\//-/g')

##### Time analysis begins:

In [27]:
echo $(date)

Thu Nov 26 00:24:04 PST 2020


## 1. Quantification of fluorescent proteins by regression

The Python package `GMOdetector` is used to quantify fluorescent proteins in each pixel of hyperspectral images via linear regression. Hyperspectral images are regressed over spectra of known components, and pixelwise maps of test-statistics are constructed for each component in the sample. This approach to quantifying components of hyperspectral images is described in-depth in the Methods section from <a href="https://link.springer.com/article/10.1007/s40789-019-0252-7" target="_blank">Böhme, et al. 2019</a>. Code and documentation for `GMOdetector` is on <a href="https://github.com/naglemi/GMOdetector_py" target="_blank">Github</a>.

In [28]:
cd $gmodetector_wd

In [29]:
job_list_name="$dataset_name.jobs"

In [30]:
rm -rf $job_list_name

In [31]:
for file in $data/*.hdr
do
 if [[ "$file" != *'hroma'* ]]; then
  echo "python wrappers/analyze_sample.py \
--file_path $file \
--fluorophores ${fluorophores[*]} \
--min_desired_wavelength ${desired_wavelength_range[0]} \
--max_desired_wavelength ${desired_wavelength_range[1]} \
--red_channel ${FalseColor_channels[0]} \
--green_channel ${FalseColor_channels[1]} \
--blue_channel ${FalseColor_channels[2]} \
--red_cap ${FalseColor_caps[0]} \
--green_cap ${FalseColor_caps[1]} \
--blue_cap ${FalseColor_caps[2]} \
--spectral_library_path "$spectral_library_path" \
--output_dir $output_directory_full" >> $job_list_name
 fi
done

In [32]:
conda activate test-environment

if [ $parallel -eq 1 ]
then
    parallel -a $job_list_name
fi

if [ $parallel -eq 0 ]
then
    bash $job_list_name
fi

conda deactivate
conda deactivate

(test-environment) (test-environment) (test-environment) (test-environment) During startup - Warning messages:
1: package ‘methods’ was built under R version 3.6.1 
2: package ‘datasets’ was built under R version 3.6.1 
3: package ‘utils’ was built under R version 3.6.1 
4: package ‘grDevices’ was built under R version 3.6.1 
5: package ‘graphics’ was built under R version 3.6.1 
6: package ‘stats’ was built under R version 3.6.1 
Running GMOdetector version 0.0.855
wrappers/analyze_sample.py:188: UserWarning: Sample is not being normalized with a chroma standard.
  warnings.warn('Sample is not being normalized with a chroma standard.')
/home/labgroup/anaconda3/envs/test-environment/lib/python3.8/site-packages/spectral/io/envi.py:175: UserWarning: Parameters with non-lowercase names encountered and converted to lowercase. To retain source file parameter name capitalization, set spectral.settings.envi_support_nonlowercase_params to True.
  warnings.warn(msg)
load mode isload_full_then_c

##### Time regression completes:

In [33]:
echo $(date)

Fri Nov 27 00:00:51 PST 2020


## 2. Neural networks to segment tissues, classify missing/contaminated explants

### 2.1. Semantic segmentation of tissues

Images are segmented into specific plant tissues by a deep neural network of the state-of-the-art Deeplab v3 architecture <a href="https://arxiv.org/abs/1706.05587" target="_blank">Liang-Chieh et al., 2017</a>. The model has been trained using training sets generated with our annotation GUI Intelligent DEep Annotator for Segmentation (IDEAS, available on <a href="https://bitbucket.org/JialinYuan/image-annotator/src/master/" target="_blank">Bitbucket</a>, publication pending). Our branch of the Deeplab v3 repo, including a Jupyter walkthrough for training, can be found on Github.

Training is completed upstream of this notebook, which only entails analysis of test data using the latest model.

<img src="Figures/downsized/segmentation_composite2.png">

Figure: This example image was taken from an experiment on the effects of different CIMs on cottonwood regeneration. This composite image illustrates that for every sample, tissues are segmented into stem (red), callus (blue) and shoot (green). These composite images, useful for manual inspection of results, are produced when the 'composite' option is on.

#### 2.1.1. Pre-processing
##### 2.1.1.1. Crop to remove labels, and resize images to 900x600

This script resizes images to 900x900 and then crops away top and bottom 150 pixels for a final image size of 900x600.

In [34]:
cd ${cwd}/intermediates/

In [35]:
conda activate base
python crop.py $data
conda deactivate

(base) (base) 

##### 2.1.1.2. prepare test.csv file for inference

Make a list of all our image files

In [36]:
cd $data
ls -d $PWD/* $data | grep -i "rgb_cropped.jpg" > test.csv

Remove the chroma standard from list of RGB image data to be segmented

In [37]:
sed -i '/hroma/d' "${data}/test.csv"

#### 2.1.2. Inference

The trained model is deployed to perform semantic segmentation of experimental images. A list of RGB images to be segmented by the trained model is passed through the --image-list option. For each of these images, we will obtain an output mask (.png) of labeled tissues

Dependencies include `opencv`, `scipy`, `yaml` and `tensorflow` (version 1.14)

In [38]:
cd $deeplab_path
conda activate deeplab
python inference.py --image-list "${data}/test.csv"
conda deactivate
conda deactivate

(deeplab) /home/labgroup/anaconda3/envs/deeplab/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/labgroup/anaconda3/envs/deeplab/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/labgroup/anaconda3/envs/deeplab/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/labgroup/anaconda3/en

#### 2.3. Post-processing

Name outputs to reflect that they are segmentation results

In [39]:
cd $data
for file in *_rgb_cropped.png; do mv -f "$file" "${file%_rgb_cropped.png}_segment_cropped.png"; done

Re-expand segment outputs to same size as original RGB files

Dependencies include `scikit-image`

In [40]:
conda activate alignment
cd $alignment_path
python expand.py $data
conda deactivate

(alignment) (alignment) Working in directory/run/media/labgroup/Elements_7//GFP_versions/GFPA_cottonwoods/wk7/Fluorescent/
Number of files: 89
Reading file GFPA1_I5.0_F1.9_L100_132600_2_0_2_segment_cropped.png
Writing file GFPA1_I5.0_F1.9_L100_132600_2_0_2_segment_uncropped.png
Reading file GFPA1_I5.0_F1.9_L100_132750_4_0_4_segment_cropped.png
Writing file GFPA1_I5.0_F1.9_L100_132750_4_0_4_segment_uncropped.png
Reading file GFPA1_I5.0_F1.9_L100_133036_7_1_6_segment_cropped.png
Writing file GFPA1_I5.0_F1.9_L100_133036_7_1_6_segment_uncropped.png
Reading file GFPA1_I5.0_F1.9_L100_133121_8_1_5_segment_cropped.png
Writing file GFPA1_I5.0_F1.9_L100_133121_8_1_5_segment_uncropped.png
Reading file GFPA1_I5.0_F1.9_L100_133216_9_1_4_segment_cropped.png
Writing file GFPA1_I5.0_F1.9_L100_133216_9_1_4_segment_uncropped.png
Reading file GFPA1_I5.0_F1.9_L100_133559_13_2_0_segment_cropped.png
Writing file GFPA1_I5.0_F1.9_L100_133559_13_2_0_segment_uncropped.png
Reading file GFPA1_I5.0_F1.9_L100_13364

Make composite images with side-by-side RGB, segmentation outputs and blended images

In [ ]:
conda activate test-environment
if [ $composite -eq 1 ]
then
    cd $gmolabeler_path
    python image_blender.py $data 0.75 'both' 1 180
fi
conda deactivate

#### 2.4. Classification of contaminated/missing explants

Plates are cropped into sub-images for each explant and each is analyzed to determine if the explant position should be excluded from analysis due to being missing or contamination. Missing and contaminated explants are recognized using a trained Densenet model (<a href="https://github.com/Contamination-Classification/DenseNet" target="_blank">Huang, et al. 2018</a>). Our fork of the Densenet repository is available on <a href="https://arxiv.org/abs/1608.06993" target="_blank">GitHub</a>.

<img src="Figures/Densenet.png">
Figure: These are four examples of contaminated explants used in the training set for this pre-trained model

If the mode for missing explant data is automatic, prepare input file for script to detect missing explants and run this script.

In [41]:
img_list_path="${data}/rgb_list.txt"

Dependencies include `keras-preprocessing`, `termcolor`,  `protobuf` and `absl-py`

In [42]:
echo $img_list_path

/run/media/labgroup/Elements_7//GFP_versions/GFPA_cottonwoods/wk7/Fluorescent//rgb_list.txt


In [43]:
missing_explants="Automatic"

In [45]:
if [ $missing_explants = "Automatic" ]; then
    conda activate densenet
    echo "Missing explants will be inferred."
    cd $data
    ls -d $PWD/* $data | grep -i "rgb.jpg" > rgb_list.txt
    sed -i '/hroma/d' rgb_list.txt
    #sed -i 's/.jpg//g' rgb_list.txt
    #cat rgb_list.txt
    cd $contamination_path
    img_list_path="${data}/rgb_list.txt"
    #echo $img_list_path
    python inference.py --img-list=$img_list_path --output_file=output.csv
    mv -f output.csv "${data}/output.csv"
    missing_explants="${data}/output.csv"
    conda deactivate
else
    echo "Missing explants input manually by user, in file: "
    echo $missing_explants
fi

Missing explants input manually by user, in file: 
/run/media/labgroup/Elements_7//GFP_versions/GFPA_cottonwoods/wk7/Fluorescent//output.csv


## 3. Alignment of RGB and hyperspectral layers

To match the frame and angle of RGB and hyperspectral image layers, we apply a scale-invariant feature transformation (<a href="https://github.com/NSF-Image-alignment/ImageAlignment" target="_blank">GitHub</a>). Using a pair of standard images, a homography matrix is calculated for the necessary transformation of RGB images to align with hyperspectral images. The transformation can then be applied to large batches of images rapidly, as long as the RGB and hyperspectral cameras remain in the same positions relative to one another (as they do in the macroPhor Array platform)

<img src="Figures/Alignment.png">
Figure: To enable precise calculation of a homography matrix for transformation of RGB images to match hyperspectral images, we used images of a piece of paper with grid marks. These images are provided by the user inputs to --hyper-img and --rgb-img in the below call to the alignment script. If using a phenotyping platform other than the macroPhor Array, or using updated camera settings, these variables will need to be replaced.

##### 3.1. Prepare input file for alignment

In [46]:
cd $data
ls | grep -i 'rgb\.jpg' > file_list_part1.csv
ls | grep -i 'segment_uncropped\.png' > file_list_part2.csv
cat file_list_part* > file_list.csv
sed -i '/hroma/d' file_list.csv
cwd2=$(pwd)/
awk -v prefix="$cwd2" '{print prefix $0}' file_list.csv > temp
mv -f temp file_list.csv
echo 'rgb_images' | cat - file_list.csv > temp && mv -f temp file_list.csv

##### 3.2 Run alignment

In [20]:
conda activate alignment
cd $alignment_path
python main.py \
--hyper-img Grids_to_align-selected/GFPA3_I5.0_F1.9_L100_142948_12_1_1_ChlorophyllLayer.csv \
--rgb-img Grids_to_align-selected/GFPA3_I5.0_F1.9_L100_142948_12_1_1_rgb.jpg \
--mode 1 \
--image_thresh_high=120 \
--image_thresh_low=50 \
--distance=0.7
conda deactivate

(alignment) (alignment) ------------Preprocess is saved and finished.-------------------
No of matches are less than 4. Cannot run RANSAC
(alignment) (base) 

: 1

In [97]:
conda activate alignment
cd $alignment_path
file_list_input="${data}/file_list.csv"
python main.py \
--hyper-img Grids_to_align-selected/12itemoffcenterleftfacinga3_F1.9_I5.0_L80_cyan_172334_0_0_0_index130.csv \
--rgb-img Grids_to_align-selected/12itemoffcenterleftfacinga3_F1.9_I5.0_L80_cyan_172334_0_0_0_rgb.jpg \
--img-csv $file_list_input \
--mode 2 \
--image_thresh_high=120 \
--image_thresh_low=50 \
--distance=0.7
conda deactivate
conda deactivate

(alignment) (alignment) (alignment) Traceback (most recent call last):
  File "main.py", line 12, in <module>
    import utils as utils
  File "/home/labgroup/code/ImageAlignment/utils.py", line 17, in <module>
    import cv2
ModuleNotFoundError: No module named 'cv2'
(alignment) 

## 4. Cross-analyze deep segmentation and regression results

Scripts in the <a href="https://github.com/naglemi/GMOlabeler" target="_blank">GMOlabeler repository</a> are used to cross-reference results from deep segmentation of RGB images and regression of hyperspectral imaging, apply thresholding parameters to classify tissues as transgenic or escapes, and produce plots.

<img src="Figures/GMOlabeler.png">

Figure: The various steps of data processing in GMOlabeler are illustrated for an example explant from an experiment on CIM optimization for cottonwood. Images of plates are cropped to a sub-image for each explant. RGB segmentation results and hyperspectral regression results are cross-referenced to calculate fluorecent proteins in specific tissues and infer whether these tissues are transgenic.

##### 4.1. Prepare sample datasheet input

Prepare input file we will use for making plots. This file contains paths to CLS results, RGB images, and hyperspectral images.

In [57]:
conda activate gmolabeler
cd "${cwd}/intermediates/"
Rscript pre_label.R \
-r "${data}/" \
-R "${output_directory_prefix}" \
-i 1 \
-d $datestamp
conda deactivate

(gmolabeler) (gmolabeler) [1] "”2020-11-26”"
[1] "Looking for CLS data in: /home/labgroup/data/gmodetector_out/Elements_7//GFP_versions/GFPA_cottonwoods/wk7/Fluorescent//"
[1] "2020-11-26"
[1] "Looking for CLS files in directory /home/labgroup/data/gmodetector_out/Elements_7//GFP_versions/GFPA_cottonwoods/wk7/Fluorescent//"
[1] "How many CLS files? 89"
[1] "We will only include in analysis the FINAL of multiple images for GFPA4_2_4"

[1] "Rows missing RGB data:"
Empty data.table (0 rows and 3 cols): segment,CLS_data,rgb

[1] "Rows missing segment data:"
Empty data.table (0 rows and 3 cols): segment,CLS_data,rgb

[1] "Rows missing regressopm data:"
                                                                                                                                             segment
1: /run/media/labgroup/Elements_7//GFP_versions/GFPA_cottonwoods/wk7/Fluorescent///GFPA4_I5.0_F1.9_L100_150752_0_2_4_segment_uncropped_processed.png
   CLS_data
1:     <NA>
                      

##### 4.2. Cross-reference RGB and hyperspectral data

In [61]:
echo $grid_file

/home/labgroup/code/ImageAlignment/112itemoffcenterleftfacinga3_F1.9_I5.0_L80_cyan_172334_0_0_0_rgb_transformed_rgb.jpg


In [92]:
conda activate gmolabeler
cd $gmolabeler_path
python main.py \
"${data}/samples_pre_labeling.csv" \
$grid_file \
$reporter_threshold \
$reporter \
$grid \
"hdf" \
$gmolabeler_path
conda deactivate

Error in certificate: Peer's certificate issuer is not recognized.
smtp-server: 535-5.7.8 Username and Password not accepted. Learn more at
smtp-server: 535 5.7.8  https://support.google.com/mail/?p=BadCredentials s10sm340065pgg.76 - gsmtp
"/home/labgroup/dead.letter" 72538/5293782
. . . message not sent.
(gmolabeler) (gmolabeler) Grid type 12
Loading plate 0 of 88
/run/media/labgroup/Elements_7//GFP_versions/GFPA_cottonwoods/wk7/Fluorescent///GFPA5_I5.0_F1.9_L100_190823_2_1_0_segment_uncropped_processed.png
Plate segment loaded
Plate RGB loaded
Loading CLS data from path/home/labgroup/data/gmodetector_out/Elements_7//GFP_versions/GFPA_cottonwoods/wk7/Fluorescent///GFPA5_I5.0_F1.9_L100_190823_2_1_0_weights.hdf
Loading plate 1 of 88
/run/media/labgroup/Elements_7//GFP_versions/GFPA_cottonwoods/wk7/Fluorescent///GFPA5_I5.0_F1.9_L100_190729_1_1_1_segment_uncropped_processed.png
Plate segment loaded
Plate RGB loaded
Loading CLS data from path/home/labgroup/data/gmodetector_out/Elements_7//

##### 4.3. Calculate sums of statistics over combined segments

We are interested in all regenerated tissue (callus + shoot) as well as all tissue (including stem as well). We will calculate aggregate statistics over these groups.

In [66]:
conda activate gmolabeler
Rscript calculate_sum_stats_over_combined_segments.R \
--output_dir "${gmolabeler_path}/output/" \
--datapath "${data_folder}/"
conda deactivate

(gmolabeler) [1] "Writing output with sums statistics calculated over combined tissue segments to: /home/labgroup/code/GMOlabeler//output/Elements_7//GFP_versions/GFPA_cottonwoods/wk7/Fluorescent//stats_with_sums_over_tissues.csv"
(gmolabeler) 

##### 4.4. Make plots of results

In [71]:
conda activate gmolabeler
cd $gmolabeler_path
Rscript grid_item_plots.R \
-d "${data_folder}/" \
-r "$randomization_datasheet" \
-p $pixel_threshold \
-v categorical \
-m 1 \
-M $missing_explants \
-g $grid \
--sort 1 \
--height $height \
--width $width
conda deactivate

(gmolabeler) (gmolabeler) Warning message:
In storage.mode(default) <- type : NAs introduced by coercion
[1] "Saving list of input arguments to : /home/labgroup/code/GMOlabeler/plots/Elements_7//GFP_versions/GFPA_cottonwoods/wk7/Fluorescent//gridplot_args.rds"
[1] "Reading in output from GMOlabeler at path: /home/labgroup/code/GMOlabeler/output/Elements_7//GFP_versions/GFPA_cottonwoods/wk7/Fluorescent//stats_with_sums_over_tissues.csv"

[1] "Rows in output from GMOlabeler: 4727"

[1] "Max n_pixels_passing_threshold in output from GMOlabeler: 12028"

[1] "Max total_signal in output from GMOlabeler: 11207608.3137862"

[1] "Look at the top of output from GMOlabeler"
[1] ""                                                                                                                     
[2] "/run/media/labgroup/Elements_7//GFP_versions/GFPA_cottonwoods/wk7/Fluorescent///GFPA5_I5.0_F1.9_L100_190823_2_1_0.png"
[3] "/run/media/labgroup/Elements_7//GFP_versions/GFPA_cottonwoods/wk7/Fluoresce

## 5. Email plots to user

##### 5.1. ZIP results

In [72]:
echo "${gmolabeler_path}/plots${data_folder}"

/home/labgroup/code/GMOlabeler//plotsElements_7//GFP_versions/GFPA_cottonwoods/wk7/Fluorescent/


In [73]:
cd "${gmolabeler_path}/plots/${data_folder}"

In [74]:
rm -f ./plants_over_plates.csv

In [75]:
cp "${gmolabeler_path}/output/${data_folder}/plants_over_plates.csv" ./

In [76]:
rm -f Rplots.pdf

In [77]:
cd ../

This messy substitution is explained here: https://superuser.com/questions/1068031/replace-backslash-with-forward-slash-in-a-variable-in-bash

In [78]:
data_folder_Compress="${data_folder////-}.zip"
data_folder_Compress=${data_folder_Compress#?};

In [79]:
ls

Fluorescent


In [80]:
cd $timepoint
zip -r $data_folder_Compress ./*

  adding: 2A1c_portion_Callus_total.png (deflated 7%)
  adding: 2A1_portion_Callus_total.png (deflated 12%)
  adding: 2A2c_portion_Callus_transgenic.png (deflated 11%)
  adding: 2A2_portion_Callus_transgenic.png (deflated 12%)
  adding: 2B1c_portion_Shoot_total.png (deflated 11%)
  adding: 2B1_portion_Shoot_total.png (deflated 13%)
  adding: 2B2c_portion_Shoot_transgenic.png (deflated 13%)
  adding: 2B2_portion_Shoot_transgenic.png (deflated 14%)
  adding: 2C1c_portion_All_regenerated_tissue_total.png (deflated 7%)
  adding: 2C1_portion_All_regenerated_tissue_total.png (deflated 12%)
  adding: 2C2c_portion_All_regenerated_tissue_transgenic.png (deflated 11%)
  adding: 2C2_portion_All_regenerated_tissue_transgenic.png (deflated 12%)
  adding: 2D1c_portion_All_tissue_total.png (deflated 10%)
  adding: 2D1_portion_All_tissue_total.png (deflated 15%)
  adding: 2D2c_portion_All_tissue_transgenic.png (deflated 9%)
  adding: 2D2_portion_All_tissue_transgenic.png (deflated 12%)
  adding: 3A1_C

##### 5.2. Write email

In [81]:
duration=$(( SECONDS - start ))

https://unix.stackexchange.com/questions/53841/how-to-use-a-timer-in-bash

In [82]:
rm -f "${gmolabeler_path}/email_to_send.txt"
cp "${gmolabeler_path}/email_to_send_template.txt" "${gmolabeler_path}/email_to_send.txt"

In [83]:
echo "" >> "${gmolabeler_path}/email_to_send.txt"
echo "Number of samples run: " >> "${gmolabeler_path}/email_to_send.txt"

In [84]:
cat "${data}/test.csv" | wc -l >> "${gmolabeler_path}/email_to_send.txt"
echo "" >> "${gmolabeler_path}/email_to_send.txt"

In [85]:
if (( $SECONDS > 3600 )) ; then
    let "hours=SECONDS/3600"
    let "minutes=(SECONDS%3600)/60"
    let "seconds=(SECONDS%3600)%60"
    echo "Completed in $hours hour(s), $minutes minute(s) and $seconds second(s)" >> "${gmolabeler_path}/email_to_send.txt"
elif (( $SECONDS > 60 )) ; then
    let "minutes=(SECONDS%3600)/60"
    let "seconds=(SECONDS%3600)%60"
    echo "Completed in $minutes minute(s) and $seconds second(s)" >> "${gmolabeler_path}/email_to_send.txt"
else
    echo "Completed in $SECONDS seconds" >> "${gmolabeler_path}/email_to_send.txt"
fi

In [86]:
echo "" >> "${gmolabeler_path}/email_to_send.txt"

##### 5.3. Send email with results to user

In [87]:
pwd

/home/labgroup/code/GMOlabeler/plots/Elements_7/GFP_versions/GFPA_cottonwoods/wk7/Fluorescent


In [88]:
mail -a $data_folder_Compress -s $data_folder $email < "${gmolabeler_path}/email_to_send.txt"

In [89]:
du -sh $data_folder_Compress

3.8M	lements_7--GFP_versions-GFPA_cottonwoods-wk7-Fluorescent-.zip


In [90]:
cat "${gmolabeler_path}/email_to_send.txt"


Your job is complete!

Number of samples run: 
89

Completed in 113 hour(s), 48 minute(s) and 29 second(s)



##### Time analysis ends

In [91]:
echo $(date)

Mon Nov 30 18:12:18 PST 2020
